# 1.5 「파인튜닝」정밀도 향상을 실현하는 방법

- pre_train된 VGG모델을 통해 미세 조정을 진행하여 개미, 꿀 이미지를 분류



# 학습목표

1.	PyTorch에서 GPU를 사용하는 코드 작성
2.	최적화 기법의 설정에서 층마다 다른 학습류을 설정한 파인튜닝 구현
3.	학습한 네트워클 저장하고 로드



In [3]:
# 함수 import
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [4]:
# 랜덤시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# Dataset과 DataLoader 작성

In [5]:
# 미리 작성한 utils 폴더의 make_dataset_dataloader.py에 기재하여 사용
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 개미와 벌의 이미지 파일 경로 리스트 작성
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Dataset 작성
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoader
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# dict 객체에 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# 네트워크 모델 설정

In [6]:
# 학습된 VGG-16 로드

# VGG-16의 인스턴스 생성
use_pretrained = True  # 학스된 파라미터 사용
vgg16_net = models.vgg16(pretrained=use_pretrained)

# VGG16의 마지막 출력층의 출력 유닛을 개미와 벌 두 개로 변경
vgg16_net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 훈련 모드로 설정
vgg16_net.train()

print('네트워크 설정 완료：학습된 가중치를 읽어들여 훈련 모드 설정 완료')


네트워크 설정 완료：학습된 가중치를 읽어들여 훈련 모드 설정 완료


# 손실함수 정의

In [7]:
# 손실함수 정의
criterion = nn.CrossEntropyLoss()

# 최적화 방법 설정

In [8]:
# 파인튜닝으로 학습할 파라미터를 params_to_update 변수의 1~3에 저장

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터명 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 파라미터를 각 리스트에 저장
for name, param in vgg16_net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장：", name)

    else:
        param.requires_grad = False
        print("경사 계산 없음. 학습하지 않음：", name)


params_to_update_1에 저장： features.0.weight
params_to_update_1에 저장： features.0.bias
params_to_update_1에 저장： features.2.weight
params_to_update_1에 저장： features.2.bias
params_to_update_1에 저장： features.5.weight
params_to_update_1에 저장： features.5.bias
params_to_update_1에 저장： features.7.weight
params_to_update_1에 저장： features.7.bias
params_to_update_1에 저장： features.10.weight
params_to_update_1에 저장： features.10.bias
params_to_update_1에 저장： features.12.weight
params_to_update_1에 저장： features.12.bias
params_to_update_1에 저장： features.14.weight
params_to_update_1에 저장： features.14.bias
params_to_update_1에 저장： features.17.weight
params_to_update_1에 저장： features.17.bias
params_to_update_1에 저장： features.19.weight
params_to_update_1에 저장： features.19.bias
params_to_update_1에 저장： features.21.weight
params_to_update_1에 저장： features.21.bias
params_to_update_1에 저장： features.24.weight
params_to_update_1에 저장： features.24.bias
params_to_update_1에 저장： features.26.weight
params_to_update_1에 저장： features.26.bias


In [9]:
# 최적화 방법 설정
# 각 파라미터마다 learning_rate를 다르게 부여
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4}, # 0.0001
    {'params': params_to_update_2, 'lr': 5e-4}, # 0.0005
    {'params': params_to_update_3, 'lr': 1e-3}  # 0.001
], momentum=0.9)


# 학습 및 검증 실시

In [10]:
# 학슴 함수 작성


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 초기 설정
    # GPU세팅 진행
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치：", device)

    # GPU 설정
    vgg16_net.to(device)

    # 네트워크가 어느 정도 고정되면 고속화시킴
    torch.backends.cudnn.benchmark = True

    # epoch 루프 진행
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                vgg16_net.train()  # 훈련 모드
            else:
                vgg16_net.eval()   # 검증 모드

            epoch_loss = 0.0  # epoch 손실 합
            epoch_corrects = 0  # epoch 정답 합

            # 학습하지 않을 때의 검증 확인을 위해、epoch=0 훈련 생략
            if (epoch == 0) and (phase == 'train'):
                continue

            # 데이터 로더에서 미니 배치를 꺼내 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPU 사용
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizer 초기화
                optimizer.zero_grad()

                # forward계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 손실 계산
                    _, preds = torch.max(outputs, 1)  # 라벨 예측

                    # 훈련 시에 bacward 진행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  # loss 합계 갱신
                    # 정답 수의 합계 갱신
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch별 loss와 정답률 표시
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [12]:
# 학습 및 검증 실행
num_epochs=2
train_model(vgg16_net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


사용 장치： cuda:0
Epoch 1/2
-------------


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.25s/it]


val Loss: 0.7704 Acc: 0.4444
Epoch 2/2
-------------


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.01s/it]


train Loss: 0.5051 Acc: 0.7325


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.03it/s]

val Loss: 0.1829 Acc: 0.9608


# 학습한 네트워크 저장 및 로드

In [14]:
# PyTorch 네트워크 파라미터 저장
save_path = './weights_fine_tuning.pth'
torch.save(vgg16_net.state_dict(), save_path)


In [16]:
# PyTorch네트워크 파라미터 로드
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
vgg16_net.load_state_dict(load_weights)

# GPU 상에 저장된 가중치를 cpu로 로드할 경우
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
vgg16_net.load_state_dict(load_weights)


<All keys matched successfully>